# Logistic regression for a multi-class problem

In this notebook, we are going to perform a logistic regression (cf. previous session slides and notebooks) on a classification problem where our classes are not just `positive` and `negative` (`pass` and `fail` in the previous notebook), but a larger range of possibilities. We are going to create a classifier identifying the dominant species of tree in a 30×30 meters patch of forest, among 7 total possible tree covers. To do so, we will rely on a much larger number of possible features than during previous classes.

## Load libraries and data

In [ ]:
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_covtype
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, classification_report

In [ ]:
dataset= fetch_covtype(data_home=os.getcwd() + "/cache/")
df = pd.DataFrame(data=dataset.data, columns=dataset.feature_names)
df["Cover_Type"] = dataset.target
df.head()

## Data exploration

### Base statistics

In [ ]:
df.describe()

In [ ]:
print("Class distribution:\n", df['Cover_Type'].value_counts())

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(14, 6))
for col, ax in zip(['Elevation','Slope','Aspect'], axes):
    ax.hist(df[col], bins=40)
    ax.set_title(f"Distribution of {col}")
    ax.set_xlabel(col)
    ax.set_ylabel("Count")

fig.tight_layout()

### Correlation matrix between continuous features

In [ ]:
continuous_features = df.columns[:10].tolist()
corr_matrix = df[continuous_features].corr()

fig, ax = plt.subplots(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5, vmin=-1, vmax=1, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode='anchor')
fig.tight_layout()

## Feature engineering

Here, we are going to create three additional features for our dataset: one computing the total hillshade of the patch, another one computing the Euclidian distance to the hydrology point, and a last one computing the interaction effect of distance to roadways and fire points.

In [ ]:
df['Hillshade_Total'] = df[['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']].sum(axis='columns')

df['Euclid_Distance_To_Hydrology'] = np.sqrt(
    df['Horizontal_Distance_To_Hydrology']**2 + df['Vertical_Distance_To_Hydrology']**2
)

df['Road_Fire_Interaction'] = df['Horizontal_Distance_To_Roadways'] * df['Horizontal_Distance_To_Fire_Points']

## Model training

Now that our dataset is ready, we are going to train our multi-class logistic regression model.

In [ ]:
X = df.drop('Cover_Type', axis=1)
y = df['Cover_Type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234, stratify=y)


In [ ]:
model = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(n_jobs=-1, max_iter=1000, penalty="l2", random_state=1234))
])

model.fit(X_train, y_train)

In [ ]:
coef_df = pd.DataFrame(model["logreg"].coef_, columns=X.columns, index=[f"Class {c}" for c in model["logreg"].classes_])

# Display top 10 predictors per class
for cls in coef_df.index:
    print(f"Top predictors for {cls}:")
    print(coef_df.loc[cls].abs().sort_values(ascending=False).head(3))
    print("\n")


## Model evaluation

In [ ]:
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
ConfusionMatrixDisplay.from_estimator(
    model,
    X_test,
    y_test,
    cmap=plt.cm.Blues
)
plt.tight_layout()

As you can see, our model performs really well for classes with a lot ef examples (1 and 2) but has trouble generalizing to the less common classes. The model design itself can also be questioned and a good strategy if we wanted to improve performances would be to try other types of models (e.g., Random Forest, Gradient Boosting) and choose the one giving the best performances.